In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import json
from cryptocompy import coin,price
import tqdm
import seaborn as sns
sns.set();

import io
import requests
import json

from datetime import datetime
import time
from time import gmtime, strftime

from sklearn import metrics
from fbprophet import Prophet
plt.rcParams["figure.figsize"] = [14,8]
pd.set_option('float_format', '{:f}'.format)

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import datetime
from datetime import datetime as dt
from datetime import timedelta


from tqdm import tqdm_notebook
from livelossplot.keras import PlotLossesCallback


from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
#from xgboost import XGBRegressor
from catboost import CatBoostRegressor, Pool, cv
import catboost
from sklearn.ensemble import RandomForestRegressor

from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
#from sklearn.cross_validation import cross_val_score
#from sklearn.grid_search import GridSearchCV
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(1)

from keras.models import Sequential  
from keras.layers.core import Dense, Activation  
from keras.layers.recurrent import LSTM
from keras.layers.advanced_activations import LeakyReLU

from keras import optimizers
from keras import backend as K

adam = optimizers.Adam(lr=0.01, clipnorm=1.)

def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 
def nn_r2_score(y_true, y_pred):
    total_error = K.sum(K.square( y_true - K.mean(y_true) ) )
    residual_error = K.sum(K.square( y_true - y_pred ))
    R_squared = 1 - (residual_error / total_error)
    return -R_squared


from math import sin, cos

from gaft import GAEngine
from gaft.components import BinaryIndividual,DecimalIndividual
from gaft.components import Population
from gaft.operators import TournamentSelection, LinearRankingSelection
from gaft.operators import UniformCrossover
from gaft.operators import FlipBitMutation
from gaft.analysis import ConsoleOutput

# Analysis plugin base class.
from gaft.plugin_interfaces.analysis import OnTheFlyAnalysis

# Built-in best fitness analysis.
from gaft.analysis.fitness_store import FitnessStore

import MySQLdb 
import sshtunnel
from sshtunnel import SSHTunnelForwarder

Using TensorFlow backend.


In [3]:
df = pd.DataFrame()
d1 = pd.DataFrame()
d2 = pd.DataFrame()
d3 = pd.DataFrame()
tech1 = pd.DataFrame()
tech2 = pd.DataFrame()
tech3 = pd.DataFrame()
tech4 = pd.DataFrame()

with SSHTunnelForwarder(('206.189.186.74', 22), ssh_password='crypto1234', ssh_username='aagam', remote_bind_address=('127.0.0.1', 3306)) as server:
    conn = MySQLdb.connect(host='localhost', port=server.local_bind_port, user='ianbalina', passwd='5!sT3jt26K%tFN*W',db='ianbalina')
    cursor = conn.cursor()
    
    #cursor.execute("SELECT * FROM ianbalina.ico_price_daily_summaries;") 
    #m = cursor.fetchone()
    query = '''SELECT * FROM ianbalina.ico_price_daily_summaries;'''
    data = pd.read_sql_query(query, conn)
    df = pd.concat([df,data])
    
    
    query = '''SELECT * FROM ianbalina.ico_grade_history;'''
    data = pd.read_sql_query(query, conn)
    d1 = pd.concat([d1,data])
    
    query = '''SELECT * FROM ianbalina.ico_analytics;'''
    data = pd.read_sql_query(query, conn)
    d2 = pd.concat([d2,data])
    
    query = '''SELECT * FROM ianbalina.tech_scorecard_answers;'''
    data = pd.read_sql_query(query, conn)
    tech1 = pd.concat([tech1,data])
    
    query = '''SELECT * FROM ianbalina.tech_scorecard_categories;'''
    data = pd.read_sql_query(query, conn)
    tech2 = pd.concat([tech2,data])
    
    query = '''SELECT * FROM ianbalina.tech_scorecard_options;'''
    data = pd.read_sql_query(query, conn)
    tech3 = pd.concat([tech3,data])
    
    query = '''SELECT * FROM ianbalina.tech_scorecard_questions;'''
    data = pd.read_sql_query(query, conn)
    tech4 = pd.concat([tech4,data])
    
    query = '''SELECT * FROM ianbalina.icos;'''
    data = pd.read_sql_query(query, conn)
    d3 = pd.concat([d3,data])
    
    
    conn.close() 

I1004 08:35:42.381536 12004 transport.py:1819] Connected (version 2.0, client OpenSSH_7.6p1)
I1004 08:35:45.168134 12004 transport.py:1819] Authentication (password) successful!


In [15]:
d1 = d1[d1['date'] == d1['date'].sort_values().iloc[-1]]

In [16]:
ids = d3['id'][d3['published_at'].notnull()].unique().tolist()
coin = []
for id in ids:
    coin.append(df['ico_symbol'][df['ico_id'] == id].unique().tolist()[0])
    
temp = pd.DataFrame()
temp['Coin'] = coin
temp['Score'] = d1['tech_scorecard_score'][d1['ico_id'].isin(ids)].values

temp = pd.DataFrame()
#d1 = d1[d1['date'] == datetime.date(2019, 9, 19)]
for coin in df['ico_symbol'].unique().tolist():
    num = df['ico_id'][df['ico_symbol'] == coin].unique().tolist()
    t = d1['tech_scorecard_score'][d1['ico_id'].isin(num)]
    if len(t)>0:
        t = t.iloc[-1]
        temp = pd.concat([temp, pd.DataFrame([coin,t]).T])

In [17]:
t2 = []
coins = []
for coin in temp['Coin'].tolist():
    t1 = df['ico_id'][df['ico_symbol'] == coin].unique().tolist()
    x = d2['all_time_growth_rate'][d2['ico_id'].isin(t1)].dropna().unique().tolist()
    if len(x) != 0:
        t2.append(x[0])
        coins.append(coin)

In [18]:
t2 = pd.DataFrame([coins,t2])
t2 = t2.T
t2.columns = ['Coin','ROI']
temp.columns = ['Coin','Score']

temp = pd.merge(temp,t2)

In [19]:
temp

,Coin,Score,ROI
0,TRX,81.818200,647.239500
1,ONT,70.909100,200.850100
2,ETH,98.181800,57982.019500
3,NEO,85.454500,3579.078100
4,BNB,90.909100,13516.003900
5,ADA,87.272700,1479.738200
6,LSK,74.545500,1031.416900
7,REP,74.545500,1288.814800
8,QTUM,67.272700,461.292900
9,OMG,74.545500,53.214800


In [20]:
temp[['Score','ROI']] = temp[['Score','ROI']].astype('float64')
'Correlation: ', temp.corr().iloc[0,1]

('Correlation: ', 0.37550466314786785)

In [21]:
temp['Score_bins'] = pd.cut(temp['Score'],[-1,10,20,30,40,50,60,70,80,90,100], right=True)
temp['ROI_bins'] = pd.cut(temp['ROI'],[-100,0,6000000])

In [23]:
final = temp['ROI_bins'].groupby([temp['Score_bins'],temp['ROI_bins']]).count().unstack()
final

ROI_bins    (-100, 0]  (0, 6000000]
Score_bins                         
(20, 30]          nan      1.000000
(30, 40]     1.000000      1.000000
(40, 50]          nan      1.000000
(50, 60]     1.000000      1.000000
(60, 70]     3.000000      1.000000
(70, 80]     3.000000      7.000000
(80, 90]     1.000000      7.000000
(90, 100]         nan      3.000000

In [24]:
x = pd.DataFrame()
x['(-100, 0]'] = [0,0] + final.values[:,0].tolist()
x['(0, 6000000]'] = [0,0] + final.values[:,1].tolist()
x.index = ['(0,10]','(10,20]','(20,30]','(30,40]','(40,50]','(50,60]','(60,70]','(70,80]','(80,90]','(90,100]']
x['Accuracy'] = [0,0,0,50,0,50, 1/4*100, 7/10*100, 7/8*100, 100]

x.fillna(0,inplace=True)

In [25]:
x

,"(-100, 0]","(0, 6000000]",Accuracy
"(0,10]",0.000000,0.000000,0.000000
"(10,20]",0.000000,0.000000,0.000000
"(20,30]",0.000000,1.000000,0.000000
"(30,40]",1.000000,1.000000,50.000000
"(40,50]",0.000000,1.000000,0.000000
"(50,60]",1.000000,1.000000,50.000000
"(60,70]",3.000000,1.000000,25.000000
"(70,80]",3.000000,7.000000,70.000000
"(80,90]",1.000000,7.000000,87.500000
"(90,100]",0.000000,3.000000,100.000000


In [15]:
tech1[tech1['ico_id'].isin(num)]

,id,tech_scorecard_question_id,ico_id,tech_scorecard_option_id,text_answer,created_at,updated_at
0,1,26,3375,69.000000,None,2019-07-28 18:28:37,2019-07-28 18:28:38
1,2,27,3375,71.000000,None,2019-07-28 18:28:37,2019-07-28 18:28:37
2,3,28,3375,73.000000,None,2019-07-28 18:28:37,2019-07-28 18:28:37
3,4,29,3375,75.000000,None,2019-07-28 18:28:40,2019-07-28 18:28:40
4,5,30,3375,77.000000,None,2019-07-28 18:28:40,2019-07-28 18:28:40
5,6,1,3375,3.000000,None,2019-07-28 18:28:44,2019-07-28 18:28:44
6,7,2,3375,6.000000,None,2019-07-28 18:28:48,2019-07-28 18:28:48
7,8,3,3375,10.000000,None,2019-07-28 18:28:50,2019-07-28 18:28:50
8,9,4,3375,12.000000,None,2019-07-28 18:28:52,2019-07-28 18:28:52
9,10,5,3375,15.000000,None,2019-07-28 18:28:58,2019-07-28 18:28:58


In [3]:
tech1

,id,tech_scorecard_question_id,ico_id,tech_scorecard_option_id,text_answer,created_at,updated_at
0,1,26,3375,69.000000,None,2019-07-28 18:28:37,2019-07-28 18:28:38
1,2,27,3375,71.000000,None,2019-07-28 18:28:37,2019-07-28 18:28:37
2,3,28,3375,73.000000,None,2019-07-28 18:28:37,2019-07-28 18:28:37
3,4,29,3375,75.000000,None,2019-07-28 18:28:40,2019-07-28 18:28:40
4,5,30,3375,77.000000,None,2019-07-28 18:28:40,2019-07-28 18:28:40
5,6,1,3375,3.000000,None,2019-07-28 18:28:44,2019-07-28 18:28:44
6,7,2,3375,6.000000,None,2019-07-28 18:28:48,2019-07-28 18:28:48
7,8,3,3375,10.000000,None,2019-07-28 18:28:50,2019-07-28 18:28:50
8,9,4,3375,12.000000,None,2019-07-28 18:28:52,2019-07-28 18:28:52
9,10,5,3375,15.000000,None,2019-07-28 18:28:58,2019-07-28 18:28:58


In [10]:
tech2

,id,title,scored,order,created_at,updated_at,short_title,binary_scale
0,1,Innovation,1,1,None,None,Innovation,nan
1,2,Architecture,1,2,None,None,Architecture,nan
2,3,Code Quality,1,3,None,None,Code Quality,nan
3,4,Mainnet,1,4,None,None,Mainnet,nan
4,5,Usability for Infrastruture Projects,1,5,None,None,Usability,nan
5,6,Team,1,6,None,None,Team,nan
6,7,References,0,7,None,None,None,nan
7,8,Initial Screening,0,0,None,None,None,1.000000
8,9,Admin Notes,0,8,None,None,None,nan
9,10,Technology Summary,0,0,None,None,None,nan


In [11]:
tech3

,id,tech_scorecard_question_id,option,score,order,created_at,updated_at
0,1,1,Bad,0.000000,1,None,None
1,2,1,Medium,1.000000,2,None,None
2,3,1,Good,2.000000,3,None,None
3,4,2,Too innovative,0.000000,1,None,None
4,5,2,Medium,1.000000,2,None,None
5,6,2,Regular,2.000000,3,None,None
6,7,3,0% crypto,0.000000,1,None,None
7,8,3,1%-5%,1.000000,2,None,None
8,9,3,6%-10%,3.000000,3,None,None
9,10,3,Over 11%,5.000000,4,None,None


In [12]:
tech4

,id,tech_scorecard_category_id,question,max_score,order,created_at,updated_at,type,visibility_private
0,1,1,How have similar projects performed?,2,1,None,None,1,0
1,2,1,Are there too many innovations?,2,2,None,None,1,0
2,3,1,Percentage of crypto users that will use the p...,5,3,None,None,1,0
3,4,1,Is the project unique?,2,4,None,None,1,0
4,5,2,Overall feeling after reading whitepaper?,2,1,None,None,1,0
5,7,2,Resistance to possible attacks?,2,2,None,None,1,0
6,8,2,Complexity of the architecture?,2,3,None,None,1,0
7,9,2,Time taken to understand the architecture?,2,4,None,None,1,0
8,10,2,Overall feeling about the architecture after d...,4,5,None,None,1,0
9,11,3,Is the project open source?,2,1,None,None,1,0
